<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/GenderRecognition/GenderRecognisionTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba

In [3]:
!unzip -qq gender-recognition-200k-images-celeba.zip

In [4]:
import os
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [5]:
width=height=224
batch_size=32

In [6]:
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
)
train_data=data_generator.flow_from_directory(
    "Dataset/Train",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
)

val_data=data_generator.flow_from_directory(
    "Dataset/Validation",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
)

test_data=data_generator.flow_from_directory(
    "Dataset/Test",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
)

Found 160000 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.
Found 20001 images belonging to 2 classes.


In [7]:
model_r=tf.keras.applications.ResNet50V2(
    input_shape=(width,height,3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

94683136/94668760 [==============================] - 1s 0us/step


In [ ]:
model_r.summary()

In [8]:
for layer in model_r.layers[:-2]:
  layer.trainable=False

In [9]:
model=tf.keras.Sequential([
        model_r,
        layers.Dense(1024,activation='relu'),
        layers.Dense(256,activation='relu'),
        layers.Dense(64,activation='relu'),
        layers.Dense(32,activation='relu'),
        layers.Dense(2,activation='softmax'),
])

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001 ),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'],)

In [11]:
model.fit(train_data,
          steps_per_epoch=train_data.samples/batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          epochs=5,
          

)

Epoch 1/5
5000/5000 [==============================] - 1129s 219ms/step - loss: 0.1592 - accuracy: 0.9365 - val_loss: 0.1437 - val_accuracy: 0.9446
Epoch 2/5
5000/5000 [==============================] - 1088s 218ms/step - loss: 0.1313 - accuracy: 0.9480 - val_loss: 0.1392 - val_accuracy: 0.9457
Epoch 3/5
5000/5000 [==============================] - 1083s 217ms/step - loss: 0.1194 - accuracy: 0.9529 - val_loss: 0.1528 - val_accuracy: 0.9460
Epoch 4/5
5000/5000 [==============================] - 1089s 218ms/step - loss: 0.1098 - accuracy: 0.9565 - val_loss: 0.1435 - val_accuracy: 0.9478
Epoch 5/5
5000/5000 [==============================] - 1089s 218ms/step - loss: 0.1018 - accuracy: 0.9600 - val_loss: 0.1350 - val_accuracy: 0.9492


In [ ]:
model.save('gender.h5')

In [12]:
model.evaluate(test_data)

626/626 [==============================] - 120s 191ms/step - loss: 0.1053 - accuracy: 0.9594


[0.10533639788627625, 0.9593520164489746]

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[11070   472]
 [  346  8113]]
Classification Report
              precision    recall  f1-score   support

      Female       0.97      0.96      0.96     11542
        Male       0.95      0.96      0.95      8459

    accuracy                           0.96     20001
   macro avg       0.96      0.96      0.96     20001
weighted avg       0.96      0.96      0.96     20001

